In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [6]:
df = pd.read_csv('2012_public_use_data_aug2016.csv')
print(df.shape)

(6720, 1119)


In [7]:
# update with new codebook created from "clean_cbecs2012_codebook.ipynb"
df_codebook = pd.read_csv('grouped_2012microdata_codebook.csv')

Index(['CATEOGORIES', 'File order', 'Variable\r\nname', 'Variable type',
       'Len-\r\ngth', 'Format', 'Label', 'Values/Format codes', 'Unnamed: 8'],
      dtype='object')


,CATEOGORIES,File order,Variable\r\nname,Variable type,Len-\r\ngth,Format,Label,Values/Format codes,Unnamed: 8
0,0,1.0,PUBID,Char,5.0,NaN,Building identifier,00001 - 06720,NaN
1,0,2.0,REGION,Char,1.0,$REGION.,Census region,'1' = 'Northeast'\r\n'2' = 'Midwest'\r\n'3' = ...,NaN
2,0,3.0,CENDIV,Char,1.0,$CENDIV.,Census division,'1' = 'New England'\r\n'2' = 'Middle Atlantic'...,NaN
3,0,4.0,PBA,Char,2.0,$PBA.,Principal building activity,'01' = 'Vacant'\r\n'02' = 'Office'\r\n'04' = '...,NaN
4,1,5.0,FREESTN,Char,1.0,$YES.,Freestanding building,1' = 'Yes'\r\nMissing='No',NaN


## EDA

Separated by Category (1-7, 10)

In [ ]:
df = df.drop(columns=category_map['get rid of'])
print(df.shape)
df.sample(5)

In [ ]:
new_1 = []
for cat_name in category_map['1']:
    if type(cat_name) == str:
        new_1.append(cat_name)
category_map['1'] = np.array(new_1)
category_map['1']

In [ ]:
dfs = {}
for cat_name, cat_list in category_map.items():
    if cat_name != 'get rid of':
        dfs[cat_name] = df[cat_list]
dfs['0'].sample(10)

In [ ]:
dfs['0'].groupby('CENDIV').size()

### EDA Category 1

Building Attributes - tried to select only continuous variables
1. Square Footage (SQFT) 1,001 - 1,500,000
2. Number of Floors (NFLOOR) 1-14, 994=15-25, 995= >25
3. Year of construction (YRCON) 995=<1946, 1946-2012
4. Any Renovations (RENOV) 1=yes, 2=no

Total Size n = 6720

Grouped by Rennovation status (`NFLOOR`, `SQFT`, `YRCON`) 
- RENOV = 1 
    - n = 3423
    - mean = 47.776220	173371.279287	1826.157172
    - median = 2	42000	1974
    - std = 206.210635	310598.178086	355.687342
- RENOV = 2 
    - n = 2936
    - mean = 11.026226	70129.489441	1883.572548
    - median = 1	10750	1985
    - std = 94.754694	174537.161246	301.333128
- RENOV Missing
    - n = 361
    
Visualizations
- I need to do some more precise manipulation to get a better idea of the spread of the data (i.e. having a hard time visualizing num floors and year constructed due to the flagged variables (994/995)

General Thoughts
- Roughly even distribution of unrennovated/rennovated buildings
- Coded data throws off averages, so better to use medians
- Older buildings without rennovations are larger (sqft/numfloor)
- Younger buildings with rennovations are smaller (sqft/numfloor)

In [ ]:
cat1_descriptions = category_dfs['1'][['col_name', 'col_description']].to_dict(orient='records')
cat1_cols_interest_descriptions = ['Square footage', 'Number of floors',  'Year of construction', 'Any renovations',]
cat1_cols_interest = []
for dictionary in cat1_descriptions:
    if dictionary['col_description'] in cat1_cols_interest_descriptions:
        cat1_cols_interest.append(dictionary['col_name'])
        print(dictionary['col_description'])
cat1_cols_interest

#SQFT Codes (Continuous)
#1,001 - 1,500,000


#NFLOOR CODES (both Continuous/Categorical)
#"1 - 14
#994 = 15 to 25
#995 = More than 25"

#YRCON CODES (Mostly Continuous)
#"995 = Before 1946
#1946 - 2012"

#RENOV CODES
#"'1' = 'Yes'
#'2' = 'No'
#Missing = Not applicable"


In [ ]:
df1 = df[cat1_cols_interest]
df1_total = df1.shape[0]
renov_group = df1.groupby('RENOV').size()
print(df1_total)
print(renov_group)
print(df1_total - (renov_group[1] + renov_group[2]))

In [ ]:
df1.pivot_table(index='RENOV', values=['SQFT', 'NFLOOR', 'YRCON'], aggfunc=np.mean)

In [ ]:
df1.pivot_table(index='RENOV', values=['SQFT', 'NFLOOR', 'YRCON'], aggfunc=np.std)

In [ ]:
df1.pivot_table(index='RENOV', values=['SQFT', 'NFLOOR', 'YRCON'], aggfunc=np.min)

In [ ]:
df1.pivot_table(index='RENOV', values=['SQFT', 'NFLOOR', 'YRCON'], aggfunc=np.max)

In [ ]:
df1.pivot_table(index='RENOV', values=['SQFT', 'NFLOOR', 'YRCON'], aggfunc=np.median)

In [ ]:
sns.histplot(df1, x='SQFT', binwidth=100000, hue='RENOV')

In [ ]:
sns.histplot(df1, x='YRCON')

### EDA Category 2

Building Ownership - Not now

In [ ]:
cat2_descriptions = category_dfs['2'][['col_name', 'col_description']].to_dict(orient='records')
cat2_descriptions

### EDA Category 3

Operations

In [ ]:
cat3_descriptions = category_dfs['3'][['col_name', 'col_description']].to_dict(orient='records')
cat3_descriptions

In [ ]:
cat3_cols_interest_descriptions = ['Months in use', 'Percent occupancy',  'Lodging room percent occupancy', 'Total hours open per week', 'Number of employees']
cat3_cols_interest = []
for dictionary in cat3_descriptions:
    if dictionary['col_description'] in cat3_cols_interest_descriptions:
        cat3_cols_interest.append(dictionary['col_name'])
        print(dictionary['col_description'])
cat3_cols_interest

# MONUSE CODES
# 0 - 12

# OCCUPYP, LODOCCP
# 0-100

# WRKHRS
# 0-168

# NWKER
# 0-6500

In [ ]:
df3 = df[cat3_cols_interest]
df3.describe()

In [ ]:
df3.groupby('MONUSE').size()

In [ ]:
df3.pivot_table(index='MONUSE', values=['OCCUPYP', 'LODOCCP', 'WKHRS', 'NWKER'], aggfunc=np.mean)

In [ ]:
df3.pivot_table(index='MONUSE', values=['OCCUPYP', 'LODOCCP', 'WKHRS', 'NWKER'], aggfunc=np.std)

In [ ]:
df3.pivot_table(index='MONUSE', values=['OCCUPYP', 'LODOCCP', 'WKHRS', 'NWKER'], aggfunc=np.min)

In [ ]:
df3.pivot_table(index='MONUSE', values=['OCCUPYP', 'LODOCCP', 'WKHRS', 'NWKER'], aggfunc=np.max)

In [ ]:
df3.pivot_table(index='MONUSE', values=['OCCUPYP', 'LODOCCP', 'WKHRS', 'NWKER'], aggfunc=np.median)

### EDA Category 4

Energy Source / Purpose

In [ ]:
cat4_descriptions = category_dfs['4'][['col_name', 'col_description']].to_dict(orient='records')
cat4_descriptions

### EDA Category 5

Energy Management

In [ ]:
cat5_descriptions = category_dfs['5'][['col_name', 'col_description']].to_dict(orient='records')
cat5_descriptions

### EDA Category 6

Service Industry Characteristics - Not right now

In [ ]:
cat6_descriptions = category_dfs['6'][['col_name', 'col_description']].to_dict(orient='records')
cat6_descriptions

### Category 7

Lighting

In [ ]:
cat7_descriptions = category_dfs['7'][['col_name', 'col_description']].to_dict(orient='records')
cat7_descriptions

### Category 10

Energy Expenses and Consumption

In [ ]:
cat10_descriptions = category_dfs['10'][['col_name', 'col_description']].to_dict(orient='records')
cat10_descriptions